In [ ]:
"""
Created on Wed Nov 22 17:27 2023

Apply weights to future as well

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

CALVING

In [ ]:
inputpath_caving = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
calving_flux = xr.open_dataset(inputpath_caving + 'calving_flux_rignot13.nc')
calving_flux_davison = xr.open_dataset(inputpath_caving + 'calving_flux_davison23.nc')

ABUMIP GL FLUX

In [ ]:
GL_flux_ABUMIP = xr.open_dataset(outputpath_GL + 'all_GL_fluxes.nc')

SMB

In [ ]:
melt_atmo_list2 = []
for mod in ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']:
    melt_atmo_list = []
    for scenario in ['historical','ssp585']:
        if scenario == 'historical':
            melt_atmo_scenario = xr.open_dataset(inputpath_atmo+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        else:
            if mod == 'IPSL-CM6A-LR':
                melt_atmo_scenario = xr.open_dataset(inputpath_atmo+mod+'_SMB_'+scenario+'-2015-2299.nc').sel(Nisf=rignot_isf)
            else:
                melt_atmo_scenario = xr.open_dataset(inputpath_atmo+mod+'_SMB_'+scenario+'-2015-2100.nc').sel(Nisf=rignot_isf)
        melt_atmo_scenario['time'] = melt_atmo_scenario['time'].dt.year

        melt_atmo_list.append(melt_atmo_scenario.assign_coords({'scenario': scenario}))
    melt_atmo = xr.concat(melt_atmo_list, dim='scenario').sel(Nisf=rignot_isf)
    melt_atmo_list2.append(melt_atmo.assign_coords({'model': mod}))
melt_atmo_all = xr.concat(melt_atmo_list2, dim='model')


OCEAN

In [ ]:
Gt_list_models = []
box1_list_models = []

for mod in ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']:
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list1 = []

    for mparam in param_classic_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp585']:
            melt_1D = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')
            melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list1.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list1, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
    box1_classic = melt1D_classic['melt_1D_mean_myr_box1']
    
    melt1D_list2 = []
    for mparam in param_NN_list:

        melt1D_scenario_list = []
        for scenario in ['historical','ssp585']:
            melt_1D = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'.nc')
            melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
        melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
        melt1D_list2.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list2, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
    box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')
    
    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    box1_all = xr.concat([box1_classic, box1_NN], dim='param')
    
    Gt_list_models.append(Gt_all.assign_coords({'model':mod}))
    box1_list_models.append(box1_all.assign_coords({'model':mod}))

Gt_all_mod = xr.concat(Gt_list_models, dim='model')
box1_all_mod = xr.concat(box1_list_models, dim='model')


    
    

MASS BALANCE

In [ ]:
mass_balance_simu = Gt_all_mod - melt_atmo_all['SMB'] + calving_flux_davison['calving_flux'].sel(time=range(1997,2015)).mean('time')

In [ ]:
bay_weights = xr.open_dataset(inputpath_weights + 'Bayesian_weights.nc')['bay_weights']

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.06,0.10,0.12,0.43,0.05]), dims=['model']).assign_coords({'model': ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']})

In [ ]:
ensmean_weighted = uf.weighted_mean(mass_balance_simu, ['model','param'], bay_weights * sens_weights)
#ensmean_nonweighted = mass_balance_simu.mean(['model','param'])

In [ ]:
da_weighted = mass_balance_simu.weighted(bay_weights * sens_weights)

In [ ]:
ensstat_weighted = da_weighted.quantile([0.05,0.5,0.95],dim=['model','param'])


In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    #melt_obs_kisf = mass_balance_obs.sel(Nisf=kisf).values
    #melt_obs_unc_kisf = melt_Gt_all_Paolo['melt_Gt_unc'].max('time').sel(Nisf=kisf).values
    #ax[i].axhline(y=melt_obs_kisf,color='k')
    #ax[i].fill_between(x=np.arange(1992,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='grey')
    
    mass_bal_hist = ensstat_weighted.sel(scenario='historical')
    mass_bal_ssp = ensstat_weighted.sel(scenario='ssp585')
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='red')
    
    ax[i].plot(np.arange(1990,2015), mass_bal_hist.sel(quantile=0.5,time=range(1990,2015),Nisf=kisf), color='blue')
    ax[i].fill_between(x=np.arange(1990,2015), y1=mass_bal_hist.sel(quantile=0.05,time=range(1990,2015),Nisf=kisf), y2=mass_bal_hist.sel(quantile=0.95,time=range(1990,2015),Nisf=kisf), color='blue',alpha=0.2)
    ax[i].plot(np.arange(2015,2101), mass_bal_ssp.sel(quantile=0.5,time=range(2015,2101),Nisf=kisf), color='blue')
    ax[i].fill_between(x=np.arange(2015,2101), y1=mass_bal_ssp.sel(quantile=0.05,time=range(2015,2101),Nisf=kisf), y2=mass_bal_ssp.sel(quantile=0.95,time=range(2015,2101),Nisf=kisf), color='blue',alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'massbalance_test_withobs_onlyChrismodels_ensmean_onlyweighted_ABUMIP.pdf')